# Логистическая регрессия. Практическая работа

## Цель практической работы

Вы уже делали задание, в котором с помощью метода ближайших соседей оценивали склонность клиента банка откликнуться или не откликнуться на предложение.

В этом практическом задании ваши цели:
*  решить эту же задачу с помощью логистической регрессии;
*  потренироваться в подборе порога; 
*  потренироваться в подборе гиперпараметров модели.

## Что входит в работу

*  Загрузить данные для задачи.
*  Обучить метод ближайших соседей с заданным количеством соседей k, вычислить метрики.
*  Обучить логистическую регрессию с параметрами по умолчанию, вычислить метрики.
*  Подобрать порог модели, вычислить метрики.
*  Подобрать гиперпараметр С (константа регуляризации) модели, вычислить метрики.

## Что оценивается

*  Выполнены все этапы задания: код запускается, отрабатывает без ошибок; подробно и обоснованно написаны текстовые выводы, где это требуется.

## Формат сдачи
Выполните предложенные задания — впишите свой код (или, если требуется, текст) в ячейки после комментариев. 

*Комментарии — это текст, который начинается с символа #. Например: # ваш код здесь.*

Сохраните изменения, используя опцию Save and Checkpoint из вкладки меню File или кнопку Save and Checkpoint на панели инструментов. Итоговый файл в формате .ipynb (файл Jupyter Notebook) загрузите в личный кабинет и отправьте на проверку.

In [1]:
# подключим библиотеки
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# считаем данные
data = pd.read_csv('ClientsData.csv')

In [3]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


В этом ноутбуке нам придётся подбирать гиперпараметры модели, а ещё порог. Поэтому, чтобы не переобучиться, разобъём данные на трейн, валидацию и тест.

*  Обучать модели будем на тренировочных данных.
*  Подбирать необходимые величины — по валидации.
*  Оценивать качество — на тесте.

In [4]:
# разделим данные на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

В задании по методу ближайших соседей было найдено, что оптимальное число соседей k = 9.

Обучите на тренировочных данных KNN с k = 9 и выведите матрицу ошибок, а также значение метрик precision и recall на тестовых данных.

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [6]:
# ваш код здесь
knn_model = KNeighborsClassifier(n_neighbors=9)

# Обучаем модель на тренировочных данных
knn_model.fit(Xtrain, ytrain)

# Предсказываем метки классов на тестовых данных
predictions = knn_model.predict(Xval)

# Вычисляем матрицу ошибок
conf_matrix = confusion_matrix(yval, predictions)
print("Confusion Matrix:")
print(conf_matrix)

# Вычисляем precision и recall
precision = precision_score(yval, predictions)
recall = recall_score(yval, predictions)

print(f"Precision: {precision}")
print(f"Recall: {recall}")

Confusion Matrix:
[[2827   17]
 [ 348    5]]
Precision: 0.22727272727272727
Recall: 0.014164305949008499


Какой вывод можно сделать:
- для класса 0 — клиент не откликнулся — мы получили достаточно высокие значения TP в том числе потому, что представителей этого класса больше;
- для класса 1 — клиент откликнулся — мы получили низкие значения TN.

Поэтому значения precision и recall низкие. Модель даёт неудовлетворительные результаты, так как находит мало клиентов, которые откликнутся на предложение.



Обучите логистическую регрессию с параметрами по умолчанию и посмотрите на метрики.

Везде дальше при оценке метрик надо выводить confusion_matrix, precision и recall.

In [7]:
from sklearn.linear_model import LogisticRegression

# ваш код здесь

logreg_model = LogisticRegression()

# Обучаем модель на тренировочных данных
logreg_model.fit(Xtrain, ytrain)

# Предсказываем метки классов на тестовых данных
pred = logreg_model.predict(Xval)

# Вычисляем матрицу ошибок
conf_matrix = confusion_matrix(yval, pred)
print("Confusion Matrix:")
print(conf_matrix)

# Вычисляем precision и recall
precision = precision_score(yval, pred)
recall = recall_score(yval, pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")

Confusion Matrix:
[[2843    1]
 [ 353    0]]
Precision: 0.0
Recall: 0.0


Наша цель — найти как можно больше клиентов, которые откликнутся на предложение. А модель таких не находит. 

Мы помним, что метод predict_proba у логистической регрессии предсказывает математические (то есть корректные) вероятности классов. Предскажите вероятности классов с помощью обученной логистической регрессии на тестовых данных и выведите вероятности положительного класса для первых десяти объектов. 

Глядя на полученные вероятности, попробуйте объяснить, почему вы получили именно такую матрицу ошибок и такие значения точности с полноты.

In [8]:
# ваш код здесь

proba_predictions = logreg_model.predict_proba(Xtest)

# Выводим вероятности положительного класса для первых десяти объектов
for i in range(10):
    print(f"Объект {i + 1}: Вероятность класса 1: {proba_predictions[i, 1]}")

Объект 1: Вероятность класса 1: 0.14042412250199765
Объект 2: Вероятность класса 1: 0.14107444773951705
Объект 3: Вероятность класса 1: 0.19378158060568823
Объект 4: Вероятность класса 1: 0.27151564120607474
Объект 5: Вероятность класса 1: 0.12612215855864198
Объект 6: Вероятность класса 1: 0.04103150633489998
Объект 7: Вероятность класса 1: 0.042618778522771426
Объект 8: Вероятность класса 1: 0.03656177747460738
Объект 9: Вероятность класса 1: 0.08229851162289745
Объект 10: Вероятность класса 1: 0.06786435300574775


In [9]:
# ваше объяснение здесь
# Думаю логистическая регрессия не может найти оптимальную линию которая бы разделяла 
# целевую переменную и по этому мы не можем получить хорошую модель

Давайте уточним цель. Пусть нам нужно найти как можно больше клиентов, которые откликнутся на предложение, то есть максимизировать полноту (recall). 

При этом хочется, чтобы точность модели (precision) не была очень низкой. Заказчик посмотрел на результаты работы KNN и потребовал, чтобы точность была не ниже, чем у KNN: $precision \geq 0.13$.

Давайте будем изменять порог для перевода вероятности в классы так, чтобы:
*   максимизировать значение recall
*   при условии, что $precision \geq 0.13$.

Если мы будем подбирать порог по тестовой выборке, то, по сути, обучимся на ней и, значит, переобучимся. Это плохо. 

Поэтому предскажите вероятности на валидационной выборке и подберите порог по ней (Xval, yval), а затем посмотрите, какое качество для найденного порога вы получите на тестовых данных.

In [10]:
from sklearn.metrics import precision_recall_curve

proba_predictions_val = logreg_model.predict_proba(Xval)[:, 1]

# Вычисляем precision, recall и thresholds
precision, recall, thresholds = precision_recall_curve(yval, proba_predictions_val)

# Находим порог, удовлетворяющий условиям задачи
desired_precision = 0.13
threshold_index = next(i for i, p in enumerate(precision) if p >= desired_precision)

selected_threshold = thresholds[threshold_index]

# Выводим найденный порог
print(f"Threshold: {selected_threshold}")

# Применяем порог к вероятностям на тестовой выборке
proba_predictions_test = logreg_model.predict_proba(Xval)[:, 1]
binary_predictions_test = (proba_predictions_test >= selected_threshold).astype(int)

# Выводим матрицу ошибок, precision и recall на тестовых данных
conf_matrix_test = confusion_matrix(yval, binary_predictions_test)
precision_test = precision_score(yval, binary_predictions_test)
recall_test = recall_score(yval, binary_predictions_test)

print("Confusion Matrix:")
print(conf_matrix_test)
print(f"Precision: {precision_test}")
print(f"Recall: {recall_test}")


Threshold: 0.0716173702888454
Confusion Matrix:
[[ 783 2061]
 [  45  308]]
Precision: 0.13001266357112706
Recall: 0.8725212464589235


In [11]:
probs_val = logreg_model.predict_proba(Xval)[:, 1]

max_recall = -1
thr = -1
prec = -1

for threshold in np.arange(0.05, 1, 0.001):
    # для каждого значения порога переведите вероятности в классы
    # посчитайте метрики

# напечатайте порог, для которого получается максимальная полнота, при precision >= 0.13

    # Переводим вероятности в бинарные предсказания по порогу
    binary_predictions_val = (probs_val >= threshold).astype(int)
    
    # Вычисляем precision и recall
    precision_val = precision_score(yval, binary_predictions_val)
    recall_val = recall_score(yval, binary_predictions_val)

    # Проверяем условия для максимизации recall и precision >= 0.13
    if recall_val > max_recall and precision_val >= 0.13:
        max_recall = recall_val
        thr = threshold
        prec = precision_val

# Печатаем порог, для которого достигается максимальная полнота, при precision >= 0.13
print(f"Selected Threshold: {thr}")
print(f"Precision: {prec}")
print(f"Recall: {max_recall}")

Selected Threshold: 0.07300000000000002
Precision: 0.1310285958173282
Recall: 0.8696883852691218


In [12]:
# по найденному порогу переведите вероятности в классы на тесте и напечатайте метрики

# Применяем найденный порог к вероятностям на тестовой выборке
binary_predictions_test = (logreg_model.predict_proba(Xtest)[:, 1] >= thr).astype(int)

# Выводим матрицу ошибок, precision и recall на тестовых данных
conf_matrix_test = confusion_matrix(ytest, binary_predictions_test)
precision_test = precision_score(ytest, binary_predictions_test)
recall_test = recall_score(ytest, binary_predictions_test)

print("Confusion Matrix:")
print(conf_matrix_test)
print(f"Precision: {precision_test}")
print(f"Recall: {recall_test}")

Confusion Matrix:
[[1152 2856]
 [ 104  455]]
Precision: 0.13742071881606766
Recall: 0.813953488372093


Сделайте вывод. Смогли ли мы с помощью подбора порога добиться большего значения recall, чем у KNN? 

In [13]:
# ваш вывод здесь
# хоть значения Recall и стало больше(в разы), но модель все еще очень плоха. Потому что даже кидая монетку можно получить такую плотность

А ещё, чтобы улучшить качество предсказания, можно подбирать значение гиперпараметра C у логистической регрессии. Для каждого значения C придётся подбирать свой порог, поэтому:  

1. Обучите для значений C из диапазона [0.05, 0.15, 0.25, ...., 10.05] логистическую регрессию (на тренировочных данных).

2. Для каждой из обученных моделей во внутреннем цикле подберите оптимальный порог (как в предыдущем задании) — на валидационных данных.



В качестве результата выведите значение C и порога для модели, которая даёт наилучшие значения метрик (наибольший recall при ограничении на $precision \geq 0.13$).

Также напечатайте полученные метрики (матрицу ошибок, точность и полноту) для лучшей модели — на тестовых данных.

In [14]:
regs = []
recalls = []
thresholds = []
precisions = []

for reg in np.arange(0.05, 10.05, 0.1):
    # Обучаем логистическую регрессию с текущим значением C
    logreg_model = LogisticRegression(C=reg)
    logreg_model.fit(Xtrain, ytrain)

    # Получаем вероятности на валидационной выборке
    probs_val = logreg_model.predict_proba(Xval)[:, 1]

    max_recall = -1
    thr = -1
    prec = -1

    for threshold in np.arange(0.05, 0.25, 0.001):
        # Переводим вероятности в бинарные предсказания по порогу
        binary_predictions_val = (probs_val >= threshold).astype(int)

        # Вычисляем precision и recall
        precision_val = precision_score(yval, binary_predictions_val)
        recall_val = recall_score(yval, binary_predictions_val)

        # Проверяем условия для максимизации recall и precision >= 0.13
        if recall_val > max_recall and precision_val >= 0.13:
            max_recall = recall_val
            thr = threshold
            prec = precision_val

    # Сохраняем результаты для текущего значения C
    regs.append(reg)
    recalls.append(max_recall)
    thresholds.append(thr)
    precisions.append(prec)

In [15]:
# выведите значения C, precision, recall, threshold для наилучшей по заданным условиям модели

# Находим индекс лучшей модели (максимизация recall при precision >= 0.13)
best_model_index = recalls.index(max(recalls))

# Выводим результаты для лучшей модели
best_C = regs[best_model_index]
best_recall = recalls[best_model_index]
best_threshold = thresholds[best_model_index]
best_precision = precisions[best_model_index]

print(f"Best C: {best_C}")
print(f"Best Threshold: {best_threshold}")
print(f"Best Precision: {best_precision}")
print(f"Best Recall: {best_recall}")

Best C: 0.05
Best Threshold: 0.07300000000000002
Best Precision: 0.1310285958173282
Best Recall: 0.8696883852691218


In [16]:
# с помощью найденных C и threshold обучите модель на тренировочных данных, сделайте предсказание на тесте и по найденному порогу получите классы
# напечатайте метрики

# Обучаем лучшую модель на тренировочных данных
best_model = LogisticRegression(C=best_C)
best_model.fit(Xtrain, ytrain)

# Получаем вероятности на тестовой выборке
proba_predictions_test = best_model.predict_proba(Xval)[:, 1]

# Применяем найденный порог к вероятностям на тестовой выборке
binary_predictions_test = (proba_predictions_test >= best_threshold).astype(int)

# Выводим матрицу ошибок, precision и recall на тестовых данных
conf_matrix_test = confusion_matrix(yval, binary_predictions_test)
precision_test = precision_score(yval, binary_predictions_test)
recall_test = recall_score(yval, binary_predictions_test)

print("\nConfusion Matrix:")
print(conf_matrix_test)
print(f"Precision: {precision_test}")
print(f"Recall: {recall_test}")


Confusion Matrix:
[[ 808 2036]
 [  46  307]]
Precision: 0.1310285958173282
Recall: 0.8696883852691218


Влияет ли изменение гиперпараметра C на качество модели (и, соответственно, метрики) в этой задаче?

In [17]:
# ваш вывод здесь
# Да, влияет, но модель все равно не сможет стать лучше. Скорее всего логистическая регрессия не подходит для этой задачи

Ответьте развёрнуто на следующие вопросы:

* Удалось ли при помощи логистической регрессии и подбора порога превзойти качество метода ближайших соседей в этой задаче? 

* Смогли ли мы при помощи этой модели получить высокий recall при ограничениях, поставленных заказчиком?


# ваш вывод здесь
При помощи логистической регрессии и подбора порога удалось превзойти качество метода ближайших соседей в этой задаче. Но желаемого результата мы не добились. Хоть и метрики стали в разы лучше, но модель не может хорошо предсказывать

Да, мы смогли при помощи этой модели получить высокий recall при ограничениях, поставленных заказчиком. Но это больше походит на самообман по вышеописанным причинам